In [2]:
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

from drn_interactions.config import ExperimentInfo
from drn_interactions.decoding.loaders import StateDecodeDataLoader
from drn_interactions.decoding.preprocessors import StateDecodePreprocessor
from drn_interactions.decoding.runners import DecodeRunner
from drn_interactions.decoding.decoders import Decoder
from drn_interactions.decoding.shuffle import shuffle_X


%load_ext autoreload
%autoreload 2

In [3]:
sessions = ExperimentInfo.eeg_sessions
session = sessions[0]
loader = StateDecodeDataLoader(block="pre", t_stop=1800, session_name=session)
preprocessor = StateDecodePreprocessor(
    thresh_empty=2,
)
spikes, states = loader()
spikes, states = preprocessor(spikes, states)

estimator = LGBMClassifier(n_estimators=50, max_depth=8, num_leaves=30)
cv = KFold(n_splits=5, shuffle=True)
decoder = Decoder(estimator=estimator, cv=cv, shuffler=shuffle_X)
runner = DecodeRunner(
    loader=loader,
    preprocessor=preprocessor,
    decoder=decoder,
    nboot=75,
)

In [5]:
pop, unit = runner.run_multiple(sessions)
limit = runner.run_multiple_limit(sessions=sessions, n_min=1, n_max=31)
dropout = runner.run_multiple_dropout(sessions=sessions, neuron_types=("SR", "SIR", "FF"))

In [9]:
from drn_interactions.config import Config


dd = Config.derived_data_dir / "decoding"

pop.to_csv(dd / "brain state - pop.csv", index=False)
unit.to_csv(dd / "brain state - unit.csv", index=False)
limit.to_csv(dd / "brain state - limit.csv", index=False)
dropout.to_csv(dd / "brain state - dropout.csv", index=False)